In [1]:
import sys, os



%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

import os
os.environ['R_HOME'] = 'D:/Program Files/R-4.5.0' #@TODO include in some setup file

from simulation_engine.scenarios.iv.binary_iv import BinaryIV
from simulation_engine.scenarios.iv.continuous_iv import ContinuousIV
from simulation_engine.scenarios.iv.base_iv import IVScenario
from simulation_engine.util.plotting_util import PlottingUtil
from simulation_engine.algorithms.causaloptim import Causaloptim
from simulation_engine.algorithms.zhang_bareinboim import ZhangBareinboim
from simulation_engine.algorithms.apid import Apid
from simulation_engine.algorithms.apid_src.src.models.apid import APID

import pandas as pd
pd.set_option('display.max_columns', None)

#disable warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from simulation_engine.algorithms.zaffalonbounds import ZaffalonBounds
from hydra import compose, initialize
from omegaconf import OmegaConf

C:\Users\tmari\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\TUM Workspace\Thesis Workspace\causal-bounds\simulation_engine\..\simulation_engine\algorithms\apid.py:233: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_name=f'config.yaml', config_path='./apid_src/config/')


In [ ]:

#generate data (or load it)
#this creates 5 simulations
import datetime


data = BinaryIV.generate_data_rolling_ate(5) 

# currently this is made for multiple simulations
# For the package this will take only one simulation
binaryIV = BinaryIV('Z->X; X->Y; U->X; X->Y', data)
runtimes = binaryIV.run(['ATE_causaloptim', 'ATE_tianpearl', 'ATE_entropybounds-0.80']) #TODO: use own parameters

Running ATE_causaloptim...
ATE_causaloptim completed in 2.70 seconds.
Running ATE_tianpearl...
ATE_tianpearl completed in 0.00 seconds.
Running ATE_entropybounds-0.80...
ATE_entropybounds-0.80 completed in 0.32 seconds.
Total runtime: 3.02 seconds.


In [9]:
binaryIV.data

,seed,intercept_X,intercept_Y,b_Z_X,b_U_X,b_X_Y,b_U_Y,ATE_true,PNS_true,p_Y1_mean,p_Y0_mean,p_U,p_Z,Z,U,X,Y,entropy_Z,entropy_U,entropy_X,entropy_Y,sigma_X,sigma_Y,squasher_X_name,squasher_Y_name,ATE_causaloptim_bound_lower,ATE_causaloptim_bound_upper,ATE_causaloptim_bound_valid,ATE_causaloptim_bound_width,ATE_causaloptim_bound_failed,ATE_tianpearl_bound_lower,ATE_tianpearl_bound_upper,ATE_tianpearl_bound_width,ATE_tianpearl_bound_failed,ATE_tianpearl_bound_valid,ATE_entropybounds-0.80_bound_lower,ATE_entropybounds-0.80_bound_upper,ATE_entropybounds-0.80_bound_valid,ATE_entropybounds-0.80_bound_width,ATE_entropybounds-0.80_bound_failed,ATE_entropybounds-0.80_theta
0,413759,0,0,1.038945,-0.081358,-5.0,-0.853965,-0.462487,0.003137,0.006095,0.468582,0.161229,0.174418,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, ...",0.619760,0.624654,0.999434,0.781481,0.249097,0.015256,sigmoid,sigmoid,-0.653373,-0.087992,True,0.565380,False,-0.945525,0.102881,1.048406,False,True,-0.742996,0.239981,True,0.982977,False,0.8
1,57609,0,0,-0.869185,0.207389,-2.5,-1.424754,-0.312326,0.027930,0.050549,0.362875,0.453207,0.491580,"[1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, ...","[1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, ...","[1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, ...","[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, ...",0.999988,0.992184,0.994904,0.781481,0.928059,0.725436,tanh_scaled,sigmoid,-0.518888,0.030384,True,0.549273,False,-0.877729,0.247232,1.124962,False,True,-0.700989,0.294817,True,0.995806,False,0.8
2,276524,0,0,0.531159,1.399690,0.0,0.711162,0.000000,0.225244,0.594666,0.594666,0.355064,0.371393,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, ...","[1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, ...","[0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, ...",0.953694,0.944331,0.939313,0.981454,0.736632,0.572904,sigmoid,probit,-0.375083,0.527157,True,0.902240,False,-0.078652,0.204969,0.283621,False,True,-0.433988,0.565988,True,0.999975,False,0.8
3,464770,0,0,0.544964,0.324027,2.5,0.129560,0.299394,0.415857,0.728616,0.429223,0.877609,0.603839,"[1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...","[1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, ...","[0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.955168,0.517753,0.784919,0.922892,0.095687,0.692420,probit,softplus,-0.239021,0.615998,True,0.855019,False,0.111111,0.456919,0.345808,False,True,-0.325001,0.675001,True,1.000001,False,0.8
4,489483,0,0,0.908082,-0.413518,5.0,-1.505112,0.587196,0.628229,0.786192,0.198996,0.842579,0.287129,"[0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, ...","[1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, ...","[1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, ...",0.873832,0.562898,0.992774,0.997402,0.569607,0.420020,sigmoid,softplus,-0.030333,0.775510,True,0.805843,False,0.636364,0.644444,0.008081,False,False,-0.355004,0.644992,True,0.999997,False,0.8
